In [1]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax



In [2]:
MODEL = f"cardiffnlp/xlm-twitter-politics-sentiment"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

text = "Good night 😊"

encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)

# # TF
# model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
# model.save_pretrained(MODEL)

# text = "Good night 😊"
# encoded_input = tokenizer(text, return_tensors='tf')
# output = model(encoded_input)
# scores = output[0][0].numpy()
# scores = softmax(scores)

# Print labels and scores
ranking = np.argsort(scores)
for i in range(scores.shape[0]):
    s = scores[ranking[i]]
    print(i, s)

0 0.0048229136
1 0.03117284
2 0.9640043


In [ ]:
import gradio as gr

gr.Interface.load("models/cardiffnlp/xlm-twitter-politics-sentiment").launch()


In [3]:
import torch
from transformers import TrainingArguments, Trainer
from sklearn.model_selection import train_test_split

# Carga y preprocesa los datos
def encode(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

# Supongamos que 'df' es tu DataFrame y que has codificado tus etiquetas como 0 y 1.
train_texts, test_texts, train_labels, test_labels = train_test_split(df['text'], df['label'], test_size=0.2)

train_encodings = encode(train_texts)
test_encodings = encode(test_texts)

class TweetDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = TweetDataset(train_encodings, train_labels)
test_dataset = TweetDataset(test_encodings, test_labels)


NameError: name 'df' is not defined